In [ ]:
#!pip install pathlib
#!pip install praw==7.5.0 

In [73]:
import pandas as pd
import os
from pathlib import Path
import numpy as np
import nltk.corpus
import spacy
import re
import string
from profanity_filter import ProfanityFilter
nlp = spacy.load('en_core_web_sm')
import warnings
warnings.filterwarnings("ignore")
import praw
from praw.models import MoreComments
pd.options.mode.chained_assignment = None

In [97]:
class Cleaner:
    def __init__(self):
        self.delete_list = ['[deleted]','[removed]','removed','deleted','https', 'Repost','repost','!gif','&#x20B','[OC]','*',
                            'instagram','facebook','twitter','google', '/r', '/u', 'http', 'www', '/s', '#']
        self.sub_list = [r'r/\S+',r'\([^)]*\)',r'http\S+',r'[()]',r'[:]',r'[\([{})\]]']
        self.pf = ProfanityFilter()
        #self.cols = ['title', 'image_link', 'subreddit']
        self.cols = ['title', 'image_link']
    
    #Removes emoticons
    def remove_emojis(data):
        emoj = re.compile("["
            u"\U0001F600-\U0001F64F"  # emoticons
            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
            u"\U0001F680-\U0001F6FF"  # transport & map symbols
            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
            u"\U00002500-\U00002BEF"  # chinese char
            u"\U00002702-\U000027B0"
            u"\U00002702-\U000027B0"
            u"\U000024C2-\U0001F251"
            u"\U0001f926-\U0001f937"
            u"\U00010000-\U0010ffff"
            u"\u2640-\u2642" 
            u"\u2600-\u2B55"
            u"\u200d"
            u"\u23cf"
            u"\u23e9"
            u"\u231a"
            u"\ufe0f"  # dingbats
            u"\u3030"
                          "]+", re.UNICODE)
        return re.sub(emoj, '', data)

    #removes characters that appear more than twice ex: booo -> boo
    keep_list = ['s','e','t','f','l','m','o','p','d','n','g','r','b']
    def remove_dupes(s):
        ans = ""
        seen = ''
        i = 0
        while i != (len(s)-1):
            if s[i] in keep_list and s[i] != seen:
                if s[i] == s[i+1]:
                    seen = s[i]
                    ans+= s[i]
                    ans+= s[i+1]
            if s[i] != seen and s[i+1] != s[i]:
                seen = s[i]
                ans += s[i]
                i += 1
            else:
                i += 1
        ans += s[len(s)-1]
        if ans[-1] == ans[-2] and ans[-1] == ans[-2]:
            ans = ans[0:-1]
        return ans

    def splitter(text, span):
        s = span
        words = re.sub(r'[^\w\s]', '', text).split(' ')
        t = [" ".join(words[i:i+s]) for i in range(0, len(words), s)]
        return t
    
    #Subsets dataset and define filters
    def clean(self, DATA, subreddit):
        data = pd.read_csv(DATA,sep=",", encoding='utf-8',on_bad_lines='skip')
        data_subset = data.drop(self.cols, axis=1)
        data_first = pd.read_csv(DATA,sep=",", encoding='utf-8',on_bad_lines='skip')[self.cols]
        for i in range(len(data_subset)):
        #for i in range(1,4):
            for j in range(len(data_subset.iloc[i])):
                text = data_subset.iloc[i][j]
                try:
                    text = remove_emojis(text)
                except:
                    continue
            
                try:
                    text = remove_dupes(text)
                except:
                    print('dupe removal failed')
                    continue
                text = text.replace("\r", " ").replace("\n", " ").replace("\t", " ").replace("_x000D_", " ").strip()
                try:
                    text = re.sub(r'\([^)]*\)', '', text)
                    for sub in self.sub_list:
                        text = re.sub(sub,'',text)
                except:
                    continue
                #text = self.pf.censor(text)
                if any(word in data_subset.iloc[i][j].lower() for word in self.delete_list):
                    text = 0
                #data_subset['comment' + str(j+1)][i] = text
                data_subset['comment' + str(j)][i] = text
            print(i,"out of",len(data_subset))
    

        #format data
        result = pd.concat([data_first, data_subset], axis=1, join='inner')
        result = result.iloc[1:, :]
        df = pd.DataFrame(columns = ['SubmissionID', 'SubmissionTitle', 'CommentID', 'Comment', 'subreddit', 'Images'])
        for i in range(len(result)):
            commentid = 1
            title = result.iloc[i][0]
            #sub = result.iloc[i][2]
            sub = subreddit
            image = result.iloc[i][1]
            for j in result.iloc[i][4:]:
                if j == 'fill':
                    continue
                elif j != 0 and len(str(j)) > 0:
                    insert = ['placeholderID', title, commentid, j, sub, image]
                    df = df.append(pd.DataFrame([insert],
                          columns = ['SubmissionID', 'SubmissionTitle', 'CommentID', 'Comment', 'subreddit', 'Images']))
                    commentid += 1
        return df

obj = Cleaner()
subreddit = input('title of csv to be cleaned:')
#data = f'data/raw_data/{subreddit}.csv'
data = f'data/raw_data/1/{subreddit}.csv'
res = obj.clean(data, subreddit)
res.to_csv(f'data/cleaned_data/{subreddit}.csv', index=False, encoding='utf8')
print('done')
print(subreddit)

title of csv to be cleaned:ThatsInsane
dupe removal failed
0 out of 196
1 out of 196
2 out of 196
3 out of 196
dupe removal failed
4 out of 196
5 out of 196
dupe removal failed
6 out of 196
7 out of 196
8 out of 196
9 out of 196
10 out of 196
dupe removal failed
dupe removal failed
11 out of 196
12 out of 196
13 out of 196
14 out of 196
15 out of 196
16 out of 196
17 out of 196
18 out of 196
19 out of 196
20 out of 196
21 out of 196
22 out of 196
dupe removal failed
23 out of 196
24 out of 196
25 out of 196
26 out of 196
dupe removal failed
27 out of 196
28 out of 196
29 out of 196
30 out of 196
31 out of 196
32 out of 196
33 out of 196
34 out of 196
35 out of 196
36 out of 196
37 out of 196
38 out of 196
39 out of 196
40 out of 196
41 out of 196
42 out of 196
43 out of 196
44 out of 196
dupe removal failed
45 out of 196
46 out of 196
47 out of 196
48 out of 196
49 out of 196
50 out of 196
51 out of 196
52 out of 196
53 out of 196
dupe removal failed
54 out of 196
dupe removal failed
5